This is a Token Classification project specific to Named Entity Recognition

1. Preparing the Data

In [1]:
import torch
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

In [2]:
from datasets import load_dataset
raw_datasets = load_dataset("lhoestq/conll2003")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


dataset_infos.json: 0.00B [00:00, ?B/s]

data/train-00000-of-00001.parquet:   0%|          | 0.00/1.07M [00:00<?, ?B/s]

data/validation-00000-of-00001.parquet:   0%|          | 0.00/281k [00:00<?, ?B/s]

data/test-00000-of-00001.parquet:   0%|          | 0.00/259k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/14041 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/3250 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/3453 [00:00<?, ? examples/s]

In [3]:
raw_datasets

DatasetDict({
    train: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 14041
    })
    validation: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 3250
    })
    test: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 3453
    })
})

In [4]:
raw_datasets["train"][0]

{'id': '0',
 'tokens': ['EU',
  'rejects',
  'German',
  'call',
  'to',
  'boycott',
  'British',
  'lamb',
  '.'],
 'pos_tags': [22, 42, 16, 21, 35, 37, 16, 21, 7],
 'chunk_tags': [11, 21, 11, 12, 21, 22, 11, 12, 0],
 'ner_tags': [3, 0, 7, 0, 0, 0, 7, 0, 0]}

In [5]:
raw_datasets["train"][0]["tokens"]

['EU', 'rejects', 'German', 'call', 'to', 'boycott', 'British', 'lamb', '.']

In [6]:
raw_datasets["train"][0]["ner_tags"]

[3, 0, 7, 0, 0, 0, 7, 0, 0]

In [7]:
ner_feature = raw_datasets["train"].features
ner_feature

{'id': Value('string'),
 'tokens': List(Value('string')),
 'pos_tags': List(Value('int64')),
 'chunk_tags': List(Value('int64')),
 'ner_tags': List(Value('int64'))}

In [8]:
# Define the NER label names yourself
label_names = [
    "O",
    "B-PER",
    "I-PER",
    "B-ORG",
    "I-ORG",
    "B-LOC",
    "I-LOC",
    "B-MISC",
    "I-MISC"
]

# Example: converting integers to labels for one sentence
example_ner_tags = [0, 3, 4, 0, 5, 6]  # integers from the dataset
example_ner_labels = [label_names[tag] for tag in example_ner_tags]

print(example_ner_labels)


['O', 'B-ORG', 'I-ORG', 'O', 'B-LOC', 'I-LOC']


In [9]:
ner_tag = raw_datasets["train"][0]["ner_tags"]
ner_tag

[3, 0, 7, 0, 0, 0, 7, 0, 0]

In [10]:
raw_datasets["train"][0]["tokens"]

['EU', 'rejects', 'German', 'call', 'to', 'boycott', 'British', 'lamb', '.']

In [11]:
new_label = [label_names[tag] for tag in ner_tag]

In [12]:
new_label

['B-ORG', 'O', 'B-MISC', 'O', 'O', 'O', 'B-MISC', 'O', 'O']

In [13]:
label_names

['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC', 'B-MISC', 'I-MISC']

In [14]:
words = raw_datasets["train"][0]["tokens"]
labels = raw_datasets["train"][0]["ner_tags"]
line1 = ""
line2 = ""
for word, label in zip(words, labels):
  full_label = label_names[label]
  max_length = max(len(word), len(full_label))
  line1 += word + " " * (max_length - len(word) + 1)
  line2 += full_label + " " * (max_length - len(full_label) + 1)

print(line1)
print(line2)

EU    rejects German call to boycott British lamb . 
B-ORG O       B-MISC O    O  O       B-MISC  O    O 


2. Data Preprocessing

In [15]:
from transformers import AutoTokenizer

model_checkpoint = "bert-base-cased"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)


tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

In [16]:
inputs = tokenizer(raw_datasets["train"][0]["tokens"], is_split_into_words=True)
inputs.word_ids()

[None, 0, 1, 2, 3, 4, 5, 6, 7, 7, 8, None]

In [17]:
def align_labels_with_tokens(labels, word_ids):
  new_labels = []
  current_word = None
  for word_id in word_ids:
    if word_id != current_word:
      current_word = word_id
      label = -100 if word_id is None else labels[word_id]
      new_labels.append(label)
    elif word_id is None:
      new_labels.append(-100)
    else:
      label = labels[word_id]
      if label % 2 ==1:
        label += 1
      new_labels.append(label)

  return new_labels

In [18]:
labels = raw_datasets["train"][0]["ner_tags"]
word_ids = inputs.word_ids()
print(labels)
print(align_labels_with_tokens(labels, word_ids))

[3, 0, 7, 0, 0, 0, 7, 0, 0]
[-100, 3, 0, 7, 0, 0, 0, 7, 0, 0, 0, -100]


In [19]:
def tokenize_and_align_labels(examples):
  tokenized_inputs = tokenizer(
      examples["tokens"], truncation=True, is_split_into_words=True
  )
  all_labels = examples["ner_tags"]
  new_labels = []
  for i, labels in enumerate(all_labels):
    word_ids = tokenized_inputs.word_ids(i)
    new_labels.append(align_labels_with_tokens(labels,word_ids))

  tokenized_inputs["labels"] = new_labels
  return tokenized_inputs

In [20]:
tokenized_datasets = raw_datasets.map(
    tokenize_and_align_labels,
    batched= True,
    remove_columns = raw_datasets["train"].column_names,
)

Map:   0%|          | 0/14041 [00:00<?, ? examples/s]

Map:   0%|          | 0/3250 [00:00<?, ? examples/s]

Map:   0%|          | 0/3453 [00:00<?, ? examples/s]

In [21]:
tokenized_datasets["train"][0]["labels"]

[-100, 3, 0, 7, 0, 0, 0, 7, 0, 0, 0, -100]

3. Fine-tuning the model with the Trainer API

In [22]:
from transformers import DataCollatorForTokenClassification

data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)

In [23]:
batch = data_collator([tokenized_datasets["train"][i] for i in range(2)])
batch["labels"]

tensor([[-100,    3,    0,    7,    0,    0,    0,    7,    0,    0,    0, -100],
        [-100,    1,    2, -100, -100, -100, -100, -100, -100, -100, -100, -100]])

In [24]:
!pip install seqeval

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 2.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16162 sha256=5570563e5ba8b723993b4f7391216565c4c15fafcfe414dde291178246a24e31
  Stored in directory: /root/.cache/pip/wheels/5f/b8/73/0b2c1a76b701a677653dd79ece07cfabd7457989dbfbdcd8d7
Successfully built seqeval


In [25]:
!pip install evaluate
import evaluate

metric = evaluate.load("seqeval")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 4.7 MB/s eta 0:00:00


In [26]:
labels = raw_datasets["train"][0]["ner_tags"]
labels = [label_names[i] for i in labels]
labels

['B-ORG', 'O', 'B-MISC', 'O', 'O', 'O', 'B-MISC', 'O', 'O']

In [27]:
predictions = labels.copy()
predictions[2] = "O"
metric.compute(predictions=[predictions], references=[labels])

{'MISC': {'precision': np.float64(1.0),
  'recall': np.float64(0.5),
  'f1': np.float64(0.6666666666666666),
  'number': np.int64(2)},
 'ORG': {'precision': np.float64(1.0),
  'recall': np.float64(1.0),
  'f1': np.float64(1.0),
  'number': np.int64(1)},
 'overall_precision': np.float64(1.0),
 'overall_recall': np.float64(0.6666666666666666),
 'overall_f1': np.float64(0.8),
 'overall_accuracy': 0.8888888888888888}

In [28]:
import numpy as np

def compute_metrics(eval_preds):
  logits, labels = eval_preds
  predictions = np.argmax(logits, axis=-1)

  true_labels = [[label_names[l] for l in label if l != -100] for label in labels]
  true_predictions = [
      [labels_names[p] for (p, l) in zip(prediction,label) if l != -100]
      for prediction, label in zip(predictions, labels)
  ]
  all_metrics = metric.compute(predictions=true_predictions, references=true_labels)
  return {
      "precision" : all_metrics["overall_precision"],
      "recall" : all_metric["overall_recall"],
      "f1" : all_metrics["overall_f1"],
      "accuracy" : all_metric["overall_accuracy"]
  }

In [29]:
id2label = {i: label for i, label in enumerate(label_names)}
label2id = {v: k for k,v in id2label.items()}

In [30]:
from transformers import AutoModelForTokenClassification

model = AutoModelForTokenClassification.from_pretrained(
    model_checkpoint,
    id2label=id2label,
    label2id=label2id
)

model.safetensors:   0%|          | 0.00/436M [00:00<?, ?B/s]

Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [31]:
model.config.num_labels

9

In [32]:
from huggingface_hub import notebook_login

notebook_login()

In [33]:
from transformers import TrainingArguments

args = TrainingArguments(
    "bert-finetuned-ner",
    # evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    num_train_epochs=3,
    weight_decay=0.01,
    push_to_hub=True
)

In [35]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=args,
    train_dataset = tokenized_datasets["train"],
    eval_dataset = tokenized_datasets["validation"],
    data_collator = data_collator,
    compute_metrics = compute_metrics,
    processing_class=tokenizer
)
trainer.train()

/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: tobilobaayodele23 (tobilobaayodele23-360learning) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Step,Training Loss
500,0.261700
1000,0.107900
1500,0.075800
2000,0.060800
2500,0.043500
3000,0.042100
3500,0.035100
4000,0.020600
4500,0.020400
5000,0.021400


TrainOutput(global_step=5268, training_loss=0.06650597033576705, metrics={'train_runtime': 694.567, 'train_samples_per_second': 60.646, 'train_steps_per_second': 7.585, 'total_flos': 920771584279074.0, 'train_loss': 0.06650597033576705, 'epoch': 3.0})

In [36]:
trainer.push_to_hub(commit_message="Training complete")

Processing Files (0 / 0)      : |          |  0.00B /  0.00B            

New Data Upload               : |          |  0.00B /  0.00B            

  ...ned-ner/training_args.bin: 100%|##########| 5.84kB / 5.84kB            

  ...439812.850331e785bc.179.0: 100%|##########| 7.83kB / 7.83kB            

  ...ned-ner/model.safetensors:   4%|3         | 16.7MB /  431MB            

No files have been modified since last commit. Skipping to prevent empty commit.


CommitInfo(commit_url='https://huggingface.co/Darkdev007/bert-finetuned-ner/commit/178c4b5d927a1cca4389751b0721b431bec4b318', commit_message='Training complete', commit_description='', oid='178c4b5d927a1cca4389751b0721b431bec4b318', pr_url=None, repo_url=RepoUrl('https://huggingface.co/Darkdev007/bert-finetuned-ner', endpoint='https://huggingface.co', repo_type='model', repo_id='Darkdev007/bert-finetuned-ner'), pr_revision=None, pr_num=None)

In [37]:
#Using a custom training loop

from torch.utils.data import DataLoader

train_dataloader = DataLoader(
    tokenized_datasets["train"],
    shuffle=True,
    collate_fn = data_collator,
    batch_size=8
)

eval_dataloader = DataLoader(
    tokenized_datasets["validation"],
    collate_fn = data_collator,
    batch_size = 8
)

In [38]:
model = AutoModelForTokenClassification.from_pretrained(
    model_checkpoint,
    id2label=id2label,
    label2id=label2id
)

Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [39]:
from torch.optim import AdamW

optimizer = AdamW(model.parameters(), lr=2e-5)

In [41]:
from accelerate import Accelerator

accelerator = Accelerator()
model, optimizer, train_dataloader, eval_dataloader = accelerator.prepare(model, optimizer, train_dataloader, eval_dataloader)

In [62]:
from transformers import get_scheduler

num_train_epochs = 3
num_training_steps = num_train_epochs * len(train_dataloader)

lr_scheduler = get_scheduler(
    "linear",
    optimizer = optimizer,
    num_warmup_steps = 0,
    num_training_steps = num_training_steps
)

In [46]:
from huggingface_hub import Repository, get_full_repo_name

model_name = "bert-finetuned-ner"
repo_name = get_full_repo_name(model_name)
repo_name

'Darkdev007/bert-finetuned-ner'

In [47]:
output_dir = "Darkdev007/bert-finetuned-ner"
repo = Repository(output_dir, clone_from=repo_name)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_deprecation.py:131: FutureWarning: 'Repository' (from 'huggingface_hub.repository') is deprecated and will be removed from version '1.0'. Please prefer the http-based alternatives instead. Given its large adoption in legacy code, the complete removal is only planned on next major release.
For more details, please read https://huggingface.co/docs/huggingface_hub/concepts/git_vs_http.
  warnings.warn(warning_message, FutureWarning)
Cloning https://huggingface.co/Darkdev007/bert-finetuned-ner into local empty directory.


Download file model.safetensors:   0%|          | 8.00k/411M [00:00<?, ?B/s]

Download file training_args.bin: 100%|##########| 5.70k/5.70k [00:00<?, ?B/s]

Download file runs/Nov28_19-49-53_ff555f026749/events.out.tfevents.1764359516.ff555f026749.559.0: 100%|#######…

Download file runs/Nov29_18-09-27_850331e785bc/events.out.tfevents.1764439812.850331e785bc.179.0: 100%|#######…

Clean file training_args.bin:  18%|#7        | 1.00k/5.70k [00:00<?, ?B/s]

Clean file runs/Nov28_19-49-53_ff555f026749/events.out.tfevents.1764359516.ff555f026749.559.0:  13%|#3        …

Clean file runs/Nov29_18-09-27_850331e785bc/events.out.tfevents.1764439812.850331e785bc.179.0:  13%|#3        …

Clean file model.safetensors:   0%|          | 1.00k/411M [00:00<?, ?B/s]

In [48]:
def postprocess(predictions, labels):
    predictions = predictions.detach().cpu().clone().numpy()
    labels = labels.detach().cpu().clone().numpy()

    # Remove ignored index (special tokens) and convert to labels
    true_labels = [[label_names[l] for l in label if l != -100] for label in labels]
    true_predictions = [
        [label_names[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    return true_labels, true_predictions

In [66]:
from tqdm.auto import tqdm
import torch

progress_bar = tqdm(range(num_training_steps))

for epoch in range(num_train_epochs):
    # Training
    model.train()
    for batch in train_dataloader:
        outputs = model(**batch)
        loss = outputs.loss
        accelerator.backward(loss)

        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        progress_bar.update(1)

    # Evaluation
    model.eval()
    for batch in eval_dataloader:
        with torch.no_grad():
            outputs = model(**batch)

        predictions = outputs.logits.argmax(dim=-1)
        labels = batch["labels"]

        # Necessary to pad predictions and labels for being gathered
        predictions = accelerator.pad_across_processes(predictions, dim=1, pad_index=-100)
        labels = accelerator.pad_across_processes(labels, dim=1, pad_index=-100)

        predictions_gathered = accelerator.gather(predictions)
        labels_gathered = accelerator.gather(labels)

        true_predictions, true_labels = postprocess(predictions_gathered, labels_gathered)
        metric.add_batch(predictions=true_predictions, references=true_labels)

    results = metric.compute()
    print(
        f"epoch {epoch}:",
        {
            key: results[f"overall_{key}"]
            for key in ["precision", "recall", "f1", "accuracy"]
        },
    )

    # Save and upload
    accelerator.wait_for_everyone()
    unwrapped_model = accelerator.unwrap_model(model)
    unwrapped_model.save_pretrained(output_dir, save_function=accelerator.save)
    if accelerator.is_main_process:
        tokenizer.save_pretrained(output_dir)
        repo.push_to_hub(
            commit_message=f"Training in progress epoch {epoch}", blocking=False
        )

  0%|          | 0/5268 [00:00<?, ?it/s]

epoch 0: {'precision': np.float64(0.9384887243352407), 'recall': np.float64(0.915455963227448), 'f1': np.float64(0.9268292682926829), 'accuracy': 0.9836198269264732}
epoch 1: {'precision': np.float64(0.9469875462807136), 'recall': np.float64(0.9250369883281275), 'f1': np.float64(0.935883575883576), 'accuracy': 0.985783246011656}
epoch 2: {'precision': np.float64(0.9469875462807136), 'recall': np.float64(0.9250369883281275), 'f1': np.float64(0.935883575883576), 'accuracy': 0.985783246011656}


Using the fine-tuned model

In [68]:
from transformers import pipeline

model_checkpoint = "Darkdev007/bert-finetuned-ner"
token_classifier = pipeline(
    "token-classification",model=model_checkpoint, aggregation_strategy="simple"
)
token_classifier("My name is Tobi and I work Hugging Face in Brooklyn.")

Device set to use cuda:0


[{'entity_group': 'PER',
  'score': np.float32(0.9879576),
  'word': 'Tobi',
  'start': 11,
  'end': 15},
 {'entity_group': 'ORG',
  'score': np.float32(0.59182984),
  'word': 'Hugging Face',
  'start': 27,
  'end': 39},
 {'entity_group': 'LOC',
  'score': np.float32(0.9966743),
  'word': 'Brooklyn',
  'start': 43,
  'end': 51}]